In [378]:
from typing import Any, Dict, List, Tuple, Set, Callable

import json
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [379]:
def interval_score(l: float, u: float, y: float, alpha: float) -> float:
    assert 0 <= alpha <= 1
    assert l <= u

    window_size: float = (u - l)
    lower_penalty: float = (2 / alpha) * (l - y) * (y < l)
    upper_penalty: float = (2 / alpha) * (y - u) * (y > u)

    score: float = window_size + lower_penalty + upper_penalty

    return score

interval_score(0.4, 4.5, 0.9, 0.1), interval_score(0.4, 4.5, 4.6, 0.1)

(4.1, 6.0999999999999925)

In [380]:
VERSION: str = "v3.0"
DATASET: str = "test"
STAGE: str = "dispatch"

CSV_PATH: str = "C:/P/backend/LambdaPY/graph_manager/builder_service/local/csv_test"

CONFIDENCE: float = 0.9

In [381]:
orders_df = pd.read_csv(f"{CSV_PATH}/order_{DATASET}.csv")
orders_df.tail()

,id,manufacturer_id,manufacturer_order_id,site_id,carrier_id,status,n_steps,tracking_link,tracking_number,manufacturer_creation_timestamp,manufacturer_estimated_delivery_timestamp,manufacturer_confirmed_delivery_timestamp,carrier_creation_timestamp,carrier_estimated_delivery_timestamp,carrier_confirmed_delivery_timestamp,SLS,SRS
5,54,1,7357,3,2,DELIVERED,12,https://www.ups.com/track?tracknum=1ZR46591041...,1ZR465910413316617,2025-05-09 11:10:00+00:00,2025-05-12 20:00:00+00:00,2025-05-16 04:47:00+00:00,2025-05-09 19:07:00+00:00,NaN,2025-05-12 13:21:00+00:00,0,0
6,100,1,8108,3,2,DELIVERED,12,https://www.ups.com/track?tracknum=1ZR46591041...,1ZR465910413368928,2025-07-11 13:39:00+00:00,2025-07-14 20:00:00+00:00,2025-07-15 08:04:00+00:00,2025-07-11 21:54:00+00:00,NaN,2025-07-15 14:59:00+00:00,0,0
7,12,1,6121,4,3,DELIVERED,12,https://www.fedex.com/fedextrack/?trknbr=43925...,439256767958,2025-01-23 15:59:00+00:00,2025-01-27 22:00:00+00:00,2025-01-28 11:58:00+00:00,2025-01-24 23:07:00+00:00,NaN,2025-01-28 11:50:00+00:00,0,0
8,60,1,7429,3,2,DELIVERED,12,https://www.ups.com/track?tracknum=1ZR46591041...,1ZR465910413322468,2025-05-16 10:01:00+00:00,2025-05-19 20:00:00+00:00,2025-05-20 06:04:00+00:00,2025-05-16 20:27:00+00:00,NaN,2025-05-19 14:28:00+00:00,0,0
9,93,1,7942,3,2,DELIVERED,11,https://www.ups.com/track?tracknum=1ZR46591041...,1ZR465910413361729,2025-07-01 10:41:00+00:00,2025-07-03 20:00:00+00:00,2025-07-15 07:08:00+00:00,2025-07-02 23:27:00+00:00,NaN,2025-07-10 13:43:00+00:00,0,0


In [382]:
estimations_path: str = f"results_final/{VERSION}/{DATASET}/estimations.json"

with open(estimations_path, "r") as f:
    estimations: Dict[str, Any] = json.load(f)

estimations[STAGE][0]

{'site': {'id': 4, 'location': 'GRAND PRAIRIE, TEXAS, US'},
 'supplier': {'id': 3, 'manufacturer_id': 3, 'name': 'MOUSER'},
 'carrier': {'id': 3, 'name': 'FedEx'},
 'manufacturer': {'id': 1,
  'name': 'FAE',
  'location': 'GAZZANIGA, LOMBARDY, IT'},
 'vertex': {'id': 21, 'name': '4', 'type': 'SUPPLIER_SITE'},
 'order_time': '2025-01-23T15:59:00+00:00',
 'shipment_time': '2025-01-24T11:18:00.001200+00:00',
 'event_time': '2025-01-23T15:59:00+00:00',
 'estimation_time': '2025-01-23T15:59:00+00:00',
 'status': 'PENDING',
 'indicators': {'parameters': {'DT': {'holidays': {'consider_closure_holidays': True,
     'consider_working_holidays': True,
     'consider_weekends_holidays': True},
    'confidence': 0.9},
   'alpha': {'type': 'EXP', 'tt_weight': 0.5},
   'PT': {'rte_estimator_params': {'model_mape': 0.4438, 'use_model': True},
    'tmi_params': {'speed_parameters': {'air_min_speed_km_h': 200.0,
      'air_max_speed_km_h': 1000.0,
      'sea_min_speed_km_h': 10.0,
      'sea_max_speed_

In [383]:
estimations[STAGE][0]['indicators']['EODT'], estimations[STAGE][0]['indicators']['CFDI']['lower'], estimations[STAGE][0]['indicators']['CFDI']['upper']

(79.17125052499999, 24.634583474999992, 24.634583474999985)

In [384]:
now = datetime.now()

(now + timedelta(hours=3) - now).days

0

In [385]:
import math

estimation_data: List[Dict[str, Any]] = []

for est in estimations[STAGE]:
    order_time: datetime = datetime.fromisoformat(est['order_time'])
    shipment_time: datetime = datetime.fromisoformat(est['shipment_time'])
    estimation_time: datetime = datetime.fromisoformat(est['estimation_time'])

    indicators: Dict[str, Any] = est['indicators']
    eodt: float = indicators['EODT']
    cfdi_lower: float = indicators['CFDI']['lower']
    cfdi_upper: float = indicators['CFDI']['upper']

    lower_bound_hours: float = eodt - cfdi_lower
    upper_bound_hours: float = eodt + cfdi_upper

    lower_bound_days: int = math.floor(lower_bound_hours / 24)
    upper_bound_days: int = math.floor(upper_bound_hours / 24)

    order_series: pd.Series = orders_df[orders_df['id'] == est['order_id']]
    delivery_time: datetime =  datetime.fromisoformat(order_series['carrier_confirmed_delivery_timestamp'].iloc[0])

    t_hours: float = (delivery_time - order_time).total_seconds() / 3600.0  # in hours
    lower_error_hours: float = max(0.0, lower_bound_hours - t_hours)
    upper_error_hours: float = max(0.0, t_hours - upper_bound_hours)

    t_day: int = math.floor((delivery_time - order_time).total_seconds() / (60 * 60 * 24))
    lower_error_day: int = max(0, lower_bound_days - t_day)
    upper_error_day: int = max(0, t_day - upper_bound_days)

    elapsed_time: float = (estimation_time - shipment_time).total_seconds() / 3600.0  # in hours
    shipment_duration: float = t_hours - (shipment_time - order_time).total_seconds() / 3600.0

    estimation_data.append({
        "order": est['order_id'],
        "elapsed_shipment_time": elapsed_time,
        "relative_elapsed_shipment_time": elapsed_time / shipment_duration,

        "window_size_hours": cfdi_lower + cfdi_upper,
        "window_size_days": upper_bound_days - lower_bound_days,

        "l_hours": lower_bound_hours,
        "u_hours": upper_bound_hours,
        "t_hours": t_hours,

        "l_days": lower_bound_days,
        "u_days": upper_bound_days,
        "t_days": t_day,

        "correct_hours": lower_bound_hours <= t_hours <= upper_bound_hours,
        "correct_days": lower_bound_days <= t_day <= upper_bound_days,

        "l_error_hours": lower_error_hours,
        "u_error_hours": upper_error_hours,

        "l_error_days": lower_error_day,
        "u_error_days": upper_error_day,
    })

estimation_df: pd.DataFrame = pd.DataFrame(estimation_data)
estimation_df

,order,elapsed_shipment_time,relative_elapsed_shipment_time,window_size_hours,window_size_days,l_hours,u_hours,t_hours,l_days,u_days,t_days,correct_hours,correct_days,l_error_hours,u_error_hours,l_error_days,u_error_days
0,12,-19.316667,-0.200104,49.269167,2,54.536667,103.805834,115.850000,2,4,4,False,True,0.000000,12.044166,0,0
1,20,-5.783333,-0.093405,78.955833,4,19.373333,98.329166,67.700000,0,4,2,True,True,0.000000,0.000000,0,0
2,44,-25.526197,-0.059424,70.201506,3,58.590674,128.792180,455.083333,2,5,18,False,False,0.000000,326.291154,0,13
3,54,-73.526197,-111.888835,70.201506,3,106.590674,176.792180,74.183333,4,7,3,False,False,32.407340,0.000000,1,0
4,60,-73.526197,-25.147454,70.201506,3,106.590674,176.792180,76.450000,4,7,3,False,False,30.140674,0.000000,1,0
5,72,-49.526197,452.167994,70.201506,3,82.590674,152.792180,49.416667,3,6,2,False,False,33.174007,0.000000,1,0
6,84,-25.526197,-0.530244,70.201506,3,58.590674,128.792180,73.666667,2,5,3,True,True,0.000000,0.000000,0,0
7,93,-25.526197,-0.131913,70.201506,3,58.590674,128.792180,219.033333,2,5,9,False,False,0.000000,90.241154,0,4
8,96,-97.526197,-2.097768,70.201506,3,130.590674,200.792180,144.016667,5,8,6,True,True,0.000000,0.000000,0,0
9,100,-73.526197,-3.088410,70.201506,3,106.590674,176.792180,97.333333,4,7,4,False,True,9.257340,0.000000,0,0


In [386]:
estimation_df["is_hours"] = estimation_df.apply(
    lambda row: interval_score(row["l_hours"], row["u_hours"], row["t_hours"], CONFIDENCE), axis=1
)
estimation_df["is_days"] = estimation_df.apply(
    lambda row: interval_score(row["l_days"], row["u_days"], row["t_days"], CONFIDENCE), axis=1
)

is_df: pd.DataFrame = pd.DataFrame(
    [
        {
            "unit": 'hours',
            "sharpness": estimation_df['window_size_hours'].mean() / 24.0,
            "coverage": estimation_df['correct_hours'].sum() / len(estimation_df),
            "mean_is": estimation_df['is_hours'].mean() / 24.0,
            "median_is": estimation_df['is_hours'].median() / 24.0,
            "std_is": estimation_df['is_hours'].std() / 24.0,
            "max_is": estimation_df['is_hours'].max() / 24.0
        },
        {
            "unit": 'days',
            "sharpness": estimation_df['window_size_days'].mean(),
            "coverage": estimation_df['correct_days'].sum() / len(estimation_df),
            "mean_is": estimation_df['is_days'].mean(),
            "median_is": estimation_df['is_days'].median(),
            "std_is": estimation_df['is_days'].std(),
            "max_is": estimation_df['is_days'].max(),
        }
    ])

is_df

,unit,sharpness,coverage,mean_is,median_is,std_is,max_is
0,hours,2.874321,0.3,7.814653,4.749045,9.253615,33.137207
1,days,3.000000,0.5,7.444444,4.611111,9.023822,31.888889


In [387]:
results_path: str = f"results_final/{VERSION}/{DATASET}/results.json"

with open(results_path, "r") as f:
    results: Dict[str, Any] = json.load(f)

results.keys()

dict_keys(['dispatch', 'shipment', 'delivery'])

In [388]:
is_dict: Dict[str, Dict[Any, Any]] = {record['unit']: record for record in is_df.to_dict(orient='records')}
for key, value in is_dict.items():
    value.pop('unit', None)

is_dict

{'hours': {'sharpness': 2.8743210435427784,
  'coverage': 0.3,
  'mean_is': 7.814652838069205,
  'median_is': 4.749044879359225,
  'std_is': 9.253614637012308,
  'max_is': 33.13720660557488},
 'days': {'sharpness': 3.0,
  'coverage': 0.5,
  'mean_is': 7.4444444444444455,
  'median_is': 4.611111111111111,
  'std_is': 9.023821545288179,
  'max_is': 31.88888888888889}}

In [389]:
results[STAGE]['overall']['interval_score'] = is_dict

with open(results_path, "w") as f:
    json.dump(results, f, indent=4)